# Spectral Classification using templates

In (Jianfeng Wu, Jonker et al 2015) they:
    
 > present Gemini spectroscopy for 21 candidate optical counterparts to X-ray sources
discovered in the Galactic Bulge Survey (GBS). 

For these sources they classify the spectra using a set of template stars. The set of template stars was chosen from the **library of the Ultraviolet and Visual Echelle Spectrograph Paranal Observatory Project** ([UVES POP](http://www.eso.org/sci/observing/tools/uvespop.html); Jehin et al. 2005), covering spectral types from A0 to M6 with luminosity class of V. The UVES POP template spectra provide coverage over the wavelength range of 3000–10000 Å with a spectral resolution of ∼ 80,000. T

> The templates were **re-sampled and Gaussian-smoothed** to match the
spectral resolution of the object spectra. The object spectra were
**Doppler-shifted into the same rest frame and averaged**. Each stellar template was optimally subtracted from the object spectrum, while a $\chi^2$ test was performed on the residuals. All the emission lines, DIBs, and telluric features (e.g., Kurucz 2006; Wallace et al. 2011) were masked during the procedure. 


# This notebook

The purpose of this noteboo kis to try to recreate what they did to be able to classify spectra for the sources I am working with. 

## A0V HD 162305

In table 4 of (Jianfeng Wu, Jonker et al 2015) they list the templates used. For the spectral type A0V they used **HD 162305** in [NGC 6475](http://simbad.u-strasbg.fr/simbad/sim-id?Ident=HD+162305&NbIdent=1&Radius=10&Radius.unit=arcmin&jsessionid=2D83A5F09A4B709E1A5077131FB78366)

In [92]:
#Import packages
from astropy.io import fits
from astropy.convolution import convolve, Gaussian1DKernel
import os
from stsci.tools import capable
capable.OF_GRAPHICS = False
from pyraf import iraf
import numpy as np
from shutil import copyfile
from astropy.convolution import convolve, Box1DKernel
#Bokeh plotting
from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
from bokeh.models import Span, Label, Arrow, NormalHead
from bokeh.models import HoverTool, tools, ColumnDataSource, CustomJS, Slider, BoxAnnotation
output_notebook()

Loading BokehJS ...

After downloading the date from http://www.eso.org/sci/observing/tools/uvespop/interface.html load the data. The data comes in the format of a fits table. 

In [93]:
t = fits.open('hd162305.tfits')
data = t[1].data
t[1].header[3:20]

NAXIS1  =                   24 / Characters in a row                            
NAXIS2  =               411803 / No. of rows in table                           
PCOUNT  =                    0 / Parameter count always 0                       
GCOUNT  =                    1 / Group count always 1                           
TFIELDS =                    3 / No. of columns in table                        
                                                                                
TFORM1  = '1D      '           / Format of field                                
TDISP1  = 'E15.5   '           / Display format of field                        
TTYPE1  = 'Wave    '           / Field label                                    
TUNIT1  = '        '           / Physical unit of field                         
TFORM2  = '1D      '           / Format of field                                
TDISP2  = 'E15.5   '           / Display format of field                        
TTYPE2  = 'Flux    '        

From the header above we know that the data has three columns: Wave, Flux and Sigma. Frim the [website](http://www.eso.org/sci/observing/tools/uvespop/legend.html#column3). 

> This final file is a tfits table with 3 columns : (1) the wavelengths rebinned to the heliocentric rest frame, (2) the flux of the combined spectra (on an arbitrary scale) and (3) the associated standard deviation.

We get the wave and flux data to plot it. 

In [94]:
waveall = data['Wave']
fluxall = data['Flux']

As we see in the header the table has 411803 rows. We dont need to plot them all. We select only each 10 element to have a first look. We plot it using Bokeh to allow for interactivity. 

In [95]:
step = 100
wavepart = wave[::step]
fluxpart = flux[::step]
source = ColumnDataSource(data=dict(x=wavepart,y=fluxpart))
hover = HoverTool(
        tooltips=[
            #("index", "$index"),
            ("(x,y)", "($x{1.11}, $y)"),
        ]
    )

plot = figure(x_axis_label='Angstrom', y_axis_label='Y',title="Spectra of an A0V: HD 162305",plot_width=900, plot_height=700)
plot.add_tools(hover)
plot.add_tools(tools.ResizeTool())
plot.line('x','y',source=source)
show(plot)

# Gaussian-smoothing

One of the steps mentioned in the paper was to do a gaussian smooth. This can be done in several ways with python. One using scipy package and the routine [scipy.ndimage.filters.gaussian_filter](https://docs.scipy.org/doc/scipy-0.15.1/reference/generated/scipy.ndimage.filters.gaussian_filter.html) or [Astropy.convolution](http://docs.astropy.org/en/stable/convolution/).

The Gaussian1Dkernel takes as required argument the standard deviation of the Gaussian kernel. 


In [97]:
# Do the smoothing
fluxgaussian10 = convolve(fluxpart, Gaussian1DKernel(stddev=10))
#Plot
source = ColumnDataSource(data=dict(x=wavepart,y=fluxgaussian10))
hover = HoverTool(
        tooltips=[
            #("index", "$index"),
            ("(x,y)", "($x{1.11}, $y)"),
        ]
    )

plot = figure(x_axis_label='Angstrom', y_axis_label='Y',title="Smoothed Spectra",plot_width=900, plot_height=700)
plot.add_tools(hover)
plot.add_tools(tools.ResizeTool())
#Eraaseplot.line(xlist,secondstar)
plot.line('x','y',source=source)
show(plot)

# References

Jianfeng Wu, P. G. Jonker, M. A. P. Torres, C. T. Britt, C. B. Johnson, R. I. Hynes, S. Greiss, D. T. H. Steeghs, T. J. Maccarone, C. O. Heinke, T. Wevers; Gemini spectroscopy of Galactic Bulge Sources: a population of hidden accreting binaries revealed?. Mon Not R Astron Soc 2015; 448 (2): 1900-1915. doi: https://doi.org/10.1093/mnras/stv047

Jehin, E., Bagnulo, S., Melo, C., Ledoux, C., & Cabanac, R. (2005). The UVES Paranal Observatory Project: a public library of high resolution stellar spectra. Proceedings of the International Astronomical Union, 1(S228), 261–262. http://doi.org/10.1017/S1743921305005739

Kurucz R. L., 2006, preprint (arXiv:https://arxiv.org/abs/astro-ph/0605029)

Wallace L., Hinkle K. H., Livingston W. C., Davis S. P., 2011, ApJS, 195, 6 http://dx.doi.org/10.1088/0067-0049/195/1/6